# Prerequisites
Before reading this tutorial you should already know Python (see the [Python Tutorial](http://docs.python.org/tut/)) and have some familiarlity with OneTick.

If you wish to work the examples in this tutorial you must also have Python installed and have followed the setup procedures for both of OneTick's pyomd and NumPy_OneTickQuery modules - instructions for which can be found with your distribution.


## Basics

The EventProcessor (EP) is the basic type in OneTick's quersive module. It is used to perform all of the analytics in OneTick, as well as to construct powerful graphs via an intuitive, pythonic interface. All EPs inherit from a single abstract class:


**EventProcessor.name**
    name of the EventProcessor

**EventProcessor.sink**
    add an event processor to this event processor as a sink.

**EventProcessor.source**:
    add an event processor to this event processor as a source.

**EventProcessor.\__lshift\__**(self,event_processor):
    overloads the >> operator and equivalent to EventProcessor.sink

**EventProcessor.\__rshift\__**(self,event_processor):
    overloads the << operator and equivalent to EventProcessor.source
    
**EventProcessor.label**
    label the event processor, returns self.
            
**EventProcessor.symbol**
    bind a symbol to the event processor   
        
**EventProcessor.pin**
    attach to the pin of the event processor (i.e., 'if','else')
        
**EventProcessor.output**
    flag the event processor as an output node
    
**===================================================================**

**EventProcessor.Descriptor**
    read-only object that describes an EventProcessor
    
**EventProcessor.Descriptor.label**
    label of the event processor
            
**EventProcessor.Descriptor.symbol**
    bound symbol of the event processor   
        
**EventProcessor.Descriptor.pin**
    input/output pin of the event processor (i.e., 'if','else')
        
**EventProcessor.Descriptor.output**
    flag to set event processor as an output node
    


The EP also provides access to a read-only nested Descriptor class to retrieve attributes describing the EP.

All of OneTick's EPs are exposed as python objects in the module. For example, the AddField EP can be instantiated like:


In [170]:
import quersive as q
a = q.add_field()
print(type(a))
isinstance(a,q.EventProcessor)

<class 'quersive.AddField'>


True

The above output demonstrates that the function call to q.add_field returns an object of type 'quersive.AddField' that inherits from the abstract quersive.EventProcessor class. Each call to an EP with no parameters passed in will populate its attributes with that EP's default values. For example, the above invocation results in the following values for the AddField object:


In [171]:
a

AddField(name='ADD_FIELD',ticktype='',field='',value='')

The above shows that the attributes of the AddField object are set to the above default values.

All of the EP's attributes are listed in its function signature. For example, OneTick's AddField EP signature is:

```python
def add_field(ticktype='', field='',value='')
```

The above signfies that, in addition to the attributes accessible via the Descriptor, each EP has it's own set of attributes as listed in its signature and can be accessed in the usual way. For example, once instantiated, the 'field' attribute of the resulting 'AddField' can be accessed and set like:


In [172]:
import quersive as q

a = q.add_field()

print(a)

#set attributes
a.ticktype='TRD'
a.field='New_Field'
a.value = 'New_Value'

print(a.ticktype)
print(a.field)
print(a.value)


try:
    a.name='New_Name'
except AttributeError as e:
    print(str(e))
    
try:
    a.descriptor = q.EventProcessor.Descriptor(label='A',symbol='B',pin='',output=True)
except AttributeError as e:
    print(str(e))

AddField(name='ADD_FIELD',ticktype='',field='',value='')
TRD
New_Field
New_Value
can't set attribute
can't set attribute


Note that the name attribute is read-only as per the EventProcessor class and can never be set, nor can the descriptor object, as shown above.The Descriptor class is read-only. To set a Descriptor field on an object, each EP provides the appropriate functions. For example, to set the label of the AddField object, call it's 'label' method. Note that the 'label' method actually returns the same AddField object, as shown below. This is useful for 'chaining' when building graphs (more below).


In [173]:
import quersive as q

a = q.add_field()

print(a.descriptor)
b = a.label('add_field')
print(a.descriptor)
(a is b)

Descriptor(output='False',symbol='',label='',pin='')
Descriptor(output='False',symbol='',label='add_field',pin='')


True

# Building Graphs - Simple Example
The below code creates a passthrough, instantiates a Graph object, and runs it via the quersive.run_query:


In [174]:
import quersive as q
from datetime import datetime

p = q.passthrough(ticktype='TRD')
g = q.Graph(p)
result = q.run_query(g,symbol='FULL_DEMO_L1::GS',s=datetime(2005,1,3,9,30),e=datetime(2005,1,3,16))
result

{'FULL_DEMO_L1::GS': [{'TICK_STATUS': array([0, 0, 0, ..., 0, 0, 0], dtype=int32), 'EXCHANGE': array(['N', 'T', 'T', ..., 'N', 'N', 'N'], 
      dtype='<U1'), 'DELETED_TIME': array(['1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000',
       '1970-01-01T00:00:00.000', ..., '1970-01-01T00:00:00.000',
       '1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000'], dtype='datetime64[ms]'), 'OMDSEQ': array([0, 0, 0, ..., 0, 0, 0], dtype=int32), 'STOP_STOCK': array(['N', 'N', 'N', ..., 'N', 'N', 'N'], 
      dtype='<U1'), 'TICKER': array(['G', 'G', 'G', ..., 'G', 'G', 'G'], 
      dtype='<U1'), 'PRICE': array([ 104.9 ,  104.9 ,  104.9 , ...,  104.85,  104.86,  104.86]), 'SEQ_NUM': array([1971504712011575, 1971504713008187, 1971504713508334, ...,
       1971527983664438, 1971527985164078, 1971527993164523], dtype=int64), 'Time': array(['2005-01-03T14:31:52.011000', '2005-01-03T14:31:53.008000',
       '2005-01-03T14:31:53.508000', ..., '2005-01-03T20:59:43.664000',
       '2005-01-03T20:59:45

Note that the output of the above is a dictionary of a list of dictionaries. All the outputs of any symbol is retrievable via the 'call' operator. This allows all outputs from queries to be retrieved. Usually, only one output is produced by the graph or we are only interested in the 'last' output of the query. As a shortcut, the [] operator retrieves the 'last' output:

In [175]:
result('FULL_DEMO_L1::GS')

[{'COND': array([' ', ' ', ' ', ..., ' ', ' ', ' '], 
        dtype='<U1'),
  'CORR': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
  'DELETED_TIME': array(['1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000',
         '1970-01-01T00:00:00.000', ..., '1970-01-01T00:00:00.000',
         '1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000'], dtype='datetime64[ms]'),
  'EXCHANGE': array(['N', 'T', 'T', ..., 'N', 'N', 'N'], 
        dtype='<U1'),
  'OMDSEQ': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
  'PRICE': array([ 104.9 ,  104.9 ,  104.9 , ...,  104.85,  104.86,  104.86]),
  'SEQ_NUM': array([1971504712011575, 1971504713008187, 1971504713508334, ...,
         1971527983664438, 1971527985164078, 1971527993164523], dtype=int64),
  'SIZE': array([40400,  1200,  1000, ...,  1200,  4900, 10000], dtype=int32),
  'SOURCE': array(['C', 'C', 'C', ..., 'C', 'C', 'C'], 
        dtype='<U1'),
  'STOP_STOCK': array(['N', 'N', 'N', ..., 'N', 'N', 'N'], 
        dtype='<U1'),
  'TICKER': array(['G

In [176]:
result['FULL_DEMO_L1::GS']

{'COND': array([' ', ' ', ' ', ..., ' ', ' ', ' '], 
       dtype='<U1'),
 'CORR': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'DELETED_TIME': array(['1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000',
        '1970-01-01T00:00:00.000', ..., '1970-01-01T00:00:00.000',
        '1970-01-01T00:00:00.000', '1970-01-01T00:00:00.000'], dtype='datetime64[ms]'),
 'EXCHANGE': array(['N', 'T', 'T', ..., 'N', 'N', 'N'], 
       dtype='<U1'),
 'OMDSEQ': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
 'PRICE': array([ 104.9 ,  104.9 ,  104.9 , ...,  104.85,  104.86,  104.86]),
 'SEQ_NUM': array([1971504712011575, 1971504713008187, 1971504713508334, ...,
        1971527983664438, 1971527985164078, 1971527993164523], dtype=int64),
 'SIZE': array([40400,  1200,  1000, ...,  1200,  4900, 10000], dtype=int32),
 'SOURCE': array(['C', 'C', 'C', ..., 'C', 'C', 'C'], 
       dtype='<U1'),
 'STOP_STOCK': array(['N', 'N', 'N', ..., 'N', 'N', 'N'], 
       dtype='<U1'),
 'TICKER': array(['G', 'G', 'G', ..., 

Note the latter produces a dict while the former produces a list of dicts, with each dict being keyed by OneTick's TickDescriptors with values as numpy arrays. This allows for rapid population of 

In [177]:
import pandas as pd

pd.DataFrame(result['FULL_DEMO_L1::GS']).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,104.9,1971504712011575,40400,C,N,G,0,2005-01-03 14:31:52.011
1,,0,1970-01-01,T,0,104.9,1971504713008187,1200,C,N,G,0,2005-01-03 14:31:53.008
2,,0,1970-01-01,T,0,104.9,1971504713508334,1000,C,N,G,0,2005-01-03 14:31:53.508
3,,0,1970-01-01,T,0,104.9,1971504713511894,100,C,N,G,0,2005-01-03 14:31:53.511
4,,0,1970-01-01,T,0,104.9,1971504713515182,200,C,N,G,0,2005-01-03 14:31:53.515


Of course, the same could be done via the 'call' operator for any of the outputs, but more verbosely:

In [178]:
pd.DataFrame(result('FULL_DEMO_L1::GS')[-1]).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,104.9,1971504712011575,40400,C,N,G,0,2005-01-03 14:31:52.011
1,,0,1970-01-01,T,0,104.9,1971504713008187,1200,C,N,G,0,2005-01-03 14:31:53.008
2,,0,1970-01-01,T,0,104.9,1971504713508334,1000,C,N,G,0,2005-01-03 14:31:53.508
3,,0,1970-01-01,T,0,104.9,1971504713511894,100,C,N,G,0,2005-01-03 14:31:53.511
4,,0,1970-01-01,T,0,104.9,1971504713515182,200,C,N,G,0,2005-01-03 14:31:53.515


# Multi-EP Graphs

Graphs can be constructed by sinking or sourcing EPs to another. To filter the above results on PRICE, for example, sink a where_clause:

In [179]:
w = q.where_clause(where='PRICE > 105')
root = p
p.sink(w)
g = q.Graph(root)
result = q.run_query(g)
pd.DataFrame(result['FULL_DEMO_L1::GS']).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,105.05,1971504952659752,1500,C,N,G,0,2005-01-03 14:35:52.659
1,E,0,1970-01-01,N,0,105.05,1971504956661036,200,C,N,G,0,2005-01-03 14:35:56.661
2,E,0,1970-01-01,N,0,105.08,1971504958660452,500,C,N,G,0,2005-01-03 14:35:58.660
3,,0,1970-01-01,N,0,105.08,1971504960661857,100,C,N,G,0,2005-01-03 14:36:00.661
4,,0,1970-01-01,N,0,105.07,1971504961161477,100,C,N,G,0,2005-01-03 14:36:01.161


The above can be achieved also via operators in place of the 'sink' or 'source' methods:

In [180]:
import quersive as q
root = p = q.passthrough(ticktype='TRD')
p >> q.where_clause(where='PRICE > 105')
g = q.Graph(root)
result = q.run_query(g,symbol='FULL_DEMO_L1::GS',s=datetime(2005,1,3,9,30),e=datetime(2005,1,3,16))
pd.DataFrame(result['FULL_DEMO_L1::GS']).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,105.05,1971504952659752,1500,C,N,G,0,2005-01-03 14:35:52.659
1,E,0,1970-01-01,N,0,105.05,1971504956661036,200,C,N,G,0,2005-01-03 14:35:56.661
2,E,0,1970-01-01,N,0,105.08,1971504958660452,500,C,N,G,0,2005-01-03 14:35:58.660
3,,0,1970-01-01,N,0,105.08,1971504960661857,100,C,N,G,0,2005-01-03 14:36:00.661
4,,0,1970-01-01,N,0,105.07,1971504961161477,100,C,N,G,0,2005-01-03 14:36:01.161


Note in the above example that we first created the passthrough EP, and then we constructed the Graph object to pass to quersive.run_query. The quesiv.Graph object is NOT and EP, but it does support sinking and sourcing of event processors. Rather than returning and EP, it'sink and source methods returns the graph:

In [181]:
import quersive as q
g = q.Graph(q.passthrough(ticktype='TRD')) >> q.where_clause(where='PRICE > 105')
result = q.run_query(g,symbol='FULL_DEMO_L1::GS',s=datetime(2005,1,3,9,30),e=datetime(2005,1,3,16))
pd.DataFrame(result['FULL_DEMO_L1::GS']).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,105.05,1971504952659752,1500,C,N,G,0,2005-01-03 14:35:52.659
1,E,0,1970-01-01,N,0,105.05,1971504956661036,200,C,N,G,0,2005-01-03 14:35:56.661
2,E,0,1970-01-01,N,0,105.08,1971504958660452,500,C,N,G,0,2005-01-03 14:35:58.660
3,,0,1970-01-01,N,0,105.08,1971504960661857,100,C,N,G,0,2005-01-03 14:36:00.661
4,,0,1970-01-01,N,0,105.07,1971504961161477,100,C,N,G,0,2005-01-03 14:36:01.161


Sourcing the where_clause can be easily achieved as well:

In [182]:
import quersive as q
g = q.Graph(q.passthrough(ticktype='TRD')) << q.where_clause(where='PRICE > 105')
result = q.run_query(g,symbol='FULL_DEMO_L1::GS',s=datetime(2005,1,3,9,30),e=datetime(2005,1,3,16))
pd.DataFrame(result['FULL_DEMO_L1::GS']).head()

,COND,CORR,DELETED_TIME,EXCHANGE,OMDSEQ,PRICE,SEQ_NUM,SIZE,SOURCE,STOP_STOCK,TICKER,TICK_STATUS,Time
0,,0,1970-01-01,N,0,105.05,1971504952659752,1500,C,N,G,0,2005-01-03 14:35:52.659
1,E,0,1970-01-01,N,0,105.05,1971504956661036,200,C,N,G,0,2005-01-03 14:35:56.661
2,E,0,1970-01-01,N,0,105.08,1971504958660452,500,C,N,G,0,2005-01-03 14:35:58.660
3,,0,1970-01-01,N,0,105.08,1971504960661857,100,C,N,G,0,2005-01-03 14:36:00.661
4,,0,1970-01-01,N,0,105.07,1971504961161477,100,C,N,G,0,2005-01-03 14:36:01.161
